In [1]:
import Nio
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

DPI = 80
FIGSIZE = (12, 6)

Сначала создадим DataFrame для данных скорости ветра.

In [2]:
file = Nio.open_file("wnd10m/wnd10m.l.gdas.197901.grb2", "r")
wind_latitude = file.variables['lat_0'][:]
wind_longitude = file.variables['lon_0'][:]
print(file.dimensions)
file.close()

{'initial_time0_hours': 124, 'forecast_time0': 7, 'lat_0': 94, 'lon_0': 192}


In [3]:
def generate_wind_dates():
    dates = []
    start_date = datetime.date(1992, 10, 1)
    finish_date = datetime.date(2018, 10, 30)
    
    while start_date <= finish_date:
        dates.append(start_date)
        start_date += datetime.timedelta(days=1)
    return dates

wind_columns = generate_wind_dates()

In [4]:
N, M = 94, 192 # lat, long
wind_rows = []
for i in range(N):
    for j in range(M):
        latitude = wind_latitude[i]
        longitude = wind_longitude[j]
        row_name = (longitude, latitude)
        wind_rows.append(str(row_name))

In [5]:
wind_df = pd.DataFrame(columns=wind_columns, index=wind_rows)

In [6]:
def generate_dates():
    start_year, finish_year = 1992, 2018
    start_month, finish_month = 10, 10
    files = []
    
    while start_year <= finish_year:
        cur_month = 1 if start_year > 1992 else start_month
        while cur_month <= 12:
            files.append(str(start_year) + str(cur_month).rjust(2, '0'))
            if (start_year == finish_year and cur_month == finish_month): break
            cur_month += 1
        start_year += 1
    
    return files

file_names = generate_dates()

In [7]:
def i_in_file_name(file_name):
    year, month = int(file_name[:4]), int(file_name[4:])
    if year > 2011 or (year == 2011 and month > 3):
        return 'i'
    return ''

def get_wind_file_name(date):
    dir_name = 'wnd10m'
    file_name_template = "wnd10m.l.gdas.{}.gr{}b2"
    file_name = file_name_template.format(date, i_in_file_name(date))   
    return str(os.path.join(dir_name, file_name))

def check_wind_file(date): 
    file = Nio.open_file(get_wind_filename(date), "r")
    d, n, m = file.variables['UGRD_P0_L103_GGA0'][2::4, 0, :, :].shape
    if (n != N or m != M):
        print(file_name, "u FAIL:", d, n, m)
    d, n, m = file.variables['VGRD_P0_L103_GGA0'][2::4, 0, :, :].shape
    if (n != N or m != M):
        print(file_name, "v FAIL:", d, n, m)
    file.close()

In [8]:
def add_file_to_csv(date):
    file_name = get_wind_file_name(date)         
    file = Nio.open_file(file_name, "r")

    cur_uw = file.variables['UGRD_P0_L103_GGA0'][2::4, 0, :, :]
    cur_vw = file.variables['VGRD_P0_L103_GGA0'][2::4, 0, :, :]
    
    year, month = int(date[:4]), int(date[4:])
    for day in range(1, cur_uw.shape[0] + 1):
        cur_date = datetime.date(year, month, day)
        coordinates_wind = []
        for i in range(N):
            for j in range(M):
                row_name = str((wind_longitude[j], wind_latitude[i]))
                cur_wind = np.sqrt(cur_uw[day - 1][i][j] ** 2 + cur_vw[day - 1][i][j] ** 2)
                wind_df.loc[row_name][cur_date] = cur_wind

    del cur_uw
    del cur_vw
    file.close()

In [9]:
for i in range(len(file_names)):
    add_file_to_csv(file_names[i])
    if i % 100 == 0:
        print(file_names[i]) # progress

199210
200906


In [10]:
wind_df

,1992-10-01,1992-10-02,1992-10-03,1992-10-04,1992-10-05,1992-10-06,1992-10-07,1992-10-08,1992-10-09,1992-10-10,...,2018-10-21,2018-10-22,2018-10-23,2018-10-24,2018-10-25,2018-10-26,2018-10-27,2018-10-28,2018-10-29,2018-10-30
"(0.0, 88.54195)",7.31095,6.60879,5.16807,3.46586,2.33206,4.68108,3.64243,7.87772,0.114018,9.14177,...,3.14884,6.80485,4.49058,2.97579,5.51983,7.68753,4.43086,3.24863,5.74952,4.90242
"(1.875, 88.54195)",7.16766,6.68001,5.17873,3.54277,2.35595,4.62935,3.70621,7.52407,0.187883,9.13731,...,3.16206,6.80312,4.49258,3.00107,5.52062,7.70019,4.46609,3.12963,5.7744,4.80803
"(3.75, 88.54195)",7.04518,6.75619,5.20419,3.63127,2.37539,4.58764,3.79585,7.18118,0.27313,9.13394,...,3.19651,6.8055,4.51108,3.03488,5.53859,7.7182,4.52356,3.03371,5.81249,4.71739
"(5.625, 88.54195)",6.92267,6.83717,5.23633,3.72414,2.40601,4.54104,3.88614,6.86236,0.354683,9.1404,...,3.2156,6.82004,4.53878,3.07262,5.55167,7.75562,4.59039,2.94061,5.86352,4.62546
"(7.5, 88.54195)",6.77835,6.9069,5.26795,3.79223,2.4233,4.49013,4.00262,6.53551,0.417852,9.13439,...,3.20975,6.81567,4.56158,3.07002,5.54525,7.74412,4.62623,2.87179,5.89702,4.55663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(350.625, -88.54195)",7.58483,6.31649,4.52027,9.72829,1.76567,3.42791,9.60277,3.39308,7.94895,6.29162,...,6.32825,6.83547,7.11805,4.68817,4.79552,4.91147,5.79146,6.96782,7.60013,7.85812
"(352.5, -88.54195)",7.58982,6.36553,4.53471,9.73358,1.81728,3.39187,9.58134,3.34964,7.96631,6.32573,...,6.37464,6.83933,7.17503,4.70134,4.80234,4.95101,5.80057,6.95018,7.58167,7.88442
"(354.375, -88.54195)",7.60497,6.41278,4.57001,9.73779,1.87182,3.36482,9.55796,3.32051,7.99218,6.36925,...,6.41955,6.85292,7.24959,4.71833,4.82001,4.99544,5.81702,6.94381,7.56086,7.92764
"(356.25, -88.54195)",7.58568,6.43747,4.56946,9.69332,1.91588,3.33542,9.51381,3.28976,7.98827,6.39806,...,6.44094,6.84877,7.31208,4.72611,4.80708,4.99308,5.80133,6.90885,7.52714,7.9422


Теперь прочитаем исходные файлы о данных аномалий уровня моря и сохраним в другой таблице.

In [11]:
file = Nio.open_file("sla/sea_level/ssh_grids_v1812_2018120512.nc", "r")
sea_longitude = file.variables['Longitude'][:]
sea_latitude = file.variables['Latitude'][:]
file.close()

In [12]:
nearest_point = dict()

for x in wind_longitude:
    for y in wind_latitude:
        cur_point = (x, y)
        nearest_x_index = (np.abs(sea_longitude - x)).argmin()
        nearest_y_index = (np.abs(sea_latitude - y)).argmin()
        nearest_point[cur_point] = (nearest_x_index, nearest_y_index)

In [13]:
sea_df = pd.DataFrame(columns=wind_df.columns, index=wind_df.index)

In [14]:
def generate_sea_dates():
    dates = []
    start_date = datetime.date(1992, 10, 2)
    finish_date = datetime.date(2018, 12, 30)
    shift = 5 # days
    
    cur_date = start_date
    while cur_date <= finish_date:
        if cur_date != datetime.date(2017, 10, 26):
            dates.append(cur_date)
        cur_date += datetime.timedelta(days=shift)
    return dates

sea_columns = generate_sea_dates()

In [15]:
def get_sea_file_name(date):
    dir_name = "sla/sea_level"
    file_name_template = "ssh_grids_v1812_{}{}{}12.nc"
    file_name = file_name_template.format(str(date[:4]), str(date[5:7]), str(date[8:]))
    return str(os.path.join(dir_name, file_name))

In [16]:
for i in range(len(wind_df.columns)):
    date = wind_df.columns[i]
    if date not in sea_columns: 
        continue
    file_name = get_sea_file_name(str(date))
    file = Nio.open_file(file_name, "r")
    cur_sea_level = file.variables['SLA'][0, :, :]

    for x in wind_longitude:
        for y in wind_latitude:
            cur_point = (x, y)
            sea_x, sea_y = nearest_point[cur_point]
            cur_sla = cur_sea_level[sea_x][sea_y]
            if isinstance(cur_sla, np.float32):
                sea_df.loc[str(cur_point)][date] = cur_sla
    if i % 1000 == 0:
        print(date)
    del cur_sea_level
    file.close()

In [17]:
use_columns = []
for column in sea_df.columns:
    if sea_df[column].isna().sum() == 18048:
        continue
    use_columns.append(column)
print(len(use_columns), len(sea_df.columns))

1903 9526


Теперь удалим ненужные столбцы и оставим только те, в которые были наблюдения (примерно каждый пятый день).

In [18]:
new_sea_df = sea_df[use_columns]
new_wind_df = wind_df[use_columns]

In [19]:
use_rows = []
for row in new_sea_df.index:
    x, y = map(float, row.replace(",", "").replace("(", "").replace(")", "").split())
    if new_sea_df.loc[row].isna().sum() != 1903 and np.abs(y) <= 80:
        use_rows.append(row)
print(len(use_rows), len(new_sea_df.index))

10237 18048


А теперь удалим лишние строки, состоящие полностью из пропусков, т.е. записи о координатах суши, а также записи о координатах с широтой по модулю больше 80. Это нужно для того, чтобы не допустить многократного использования один и тех же значения, т.к. исходные данные аномалий уровня моря содержат данные по широте в пределах [-80..80].

In [20]:
cropped_sea_df = new_sea_df.loc[use_rows]
cropped_wind_df = new_wind_df.loc[use_rows]

In [21]:
cropped_sea_df.to_csv("sla_2.csv")
cropped_wind_df.to_csv("wind_2.csv")